# FIFA 21 MoneyBall

Use a Linear Regression Model to predict the Value of FIFA players.

## Import Libraries

In [18]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
from scipy.stats.mstats import winsorize

import math
from sklearn.preprocessing import OneHotEncoder, Normalizer, LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.neighbors import KNeighborsRegressor

import warnings
warnings.filterwarnings('ignore')

## Import Dataset

In [19]:
data = pd.read_csv(r"Data/fifa21_male2.csv")

## Review and Clean Data

In [20]:
data.shape

(17125, 107)

In [21]:
data.head()

,ID,Name,Age,OVA,Nationality,Club,BOV,BP,Position,Player Photo,Club Logo,Flag Photo,POT,Team & Contract,Height,Weight,foot,Growth,Joined,Loan Date End,Value,Wage,Release Clause,Contract,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,Gender
0,2,G. Pasquale,33,69,Italy,Udinese,71,LWB,LM,https://cdn.sofifa.com/players/000/002/16_120.png,https://cdn.sofifa.com/teams/55/light_60.png,https://cdn.sofifa.com/flags/it.png,69,Udinese 2008 ~ 2016,"6'0""",181lbs,Left,0,"Jul 1, 2008",NaN,€625K,€7K,€0,2008 ~ 2016,313,75,50,59,71,58.0,338,73,65.0,60,69,71,347,68,74,68.0,69,68.0,347,74,68.0,69,68,68,320,72,69.0,63.0,66.0,50,NaN,208,70,69,69.0,56,14,5,15,10,12,1929,408,3 ★,2★,Medium,High,2 ★,71,59,70,71,68,69,4,65+0,65+0,65+0,68+0,67+0,67+0,67+0,68+0,68+0,68+0,68+0,69+0,69+0,69+0,69+0,69+0,71+-2,70+-1,70+-1,70+-1,71+-2,70+-1,69+0,69+0,69+0,70+-1,17+0,Male
1,16,Luis García,37,71,Spain,KAS Eupen,70,CM,CM CAM CDM,https://cdn.sofifa.com/players/000/016/19_120.png,https://cdn.sofifa.com/teams/2013/light_60.png,https://cdn.sofifa.com/flags/es.png,71,KAS Eupen 2014 ~ 2019,"5'10""",143lbs,Right,0,"Jul 19, 2014",NaN,€600K,€7K,€1.1M,2014 ~ 2019,337,68,64,61,76,68.0,369,69,79.0,79,71,71,305,56,50,62.0,65,72.0,324,75,54.0,64,60,71,362,71,71.0,72.0,73.0,75,79.0,153,70,43,40.0,56,9,12,13,11,11,1906,385,4 ★,3★,Medium,Medium,1 ★,53,69,73,69,58,63,4,67+1,67+1,67+1,67+0,68+0,68+0,68+0,67+0,70+1,70+1,70+1,68+1,70+1,70+1,70+1,68+1,62+1,66+1,66+1,66+1,62+1,60+1,60+1,60+1,60+1,60+1,17+1,Male
2,27,J. Cole,33,71,England,Coventry City,71,CAM,CAM RM RW LM,https://cdn.sofifa.com/players/000/027/16_120.png,https://cdn.sofifa.com/teams/1800/light_60.png,https://cdn.sofifa.com/flags/gb-eng.png,71,Coventry City 2016 ~ 2020,"5'9""",161lbs,Right,0,"Jan 7, 2016",NaN,€1.1M,€15K,€0,2016 ~ 2020,337,80,64,41,77,75.0,387,79,84.0,77,69,78,295,48,42,71.0,59,75.0,284,72,58.0,29,56,69,317,69,39.0,69.0,74.0,66,NaN,99,35,34,30.0,51,9,6,13,16,7,1770,354,4 ★,4★,Medium,Low,2 ★,45,68,76,77,36,52,11,64+0,64+0,64+0,70+0,69+0,69+0,69+0,70+0,71+0,71+0,71+0,68+0,66+0,66+0,66+0,68+0,52+0,54+0,54+0,54+0,52+0,47+0,46+0,46+0,46+0,47+0,15+0,Male
3,36,D. Yorke,36,68,Trinidad &amp; Tobago,Sunderland,70,ST,NaN,https://cdn.sofifa.com/players/000/036/09_120.png,https://cdn.sofifa.com/teams/106/light_60.png,https://cdn.sofifa.com/flags/tt.png,82,Sunderland 2009,"5'11""",165lbs,Right,14,NaN,NaN,€0,€0,€0,2009,264,54,70,60,80,NaN,255,68,NaN,46,64,77,176,59,62,NaN,55,NaN,239,63,NaN,51,66,59,271,59,70.0,72.0,NaN,70,NaN,75,34,41,NaN,68,5,21,64,21,21,1348,369,3 ★,1★,NaN,NaN,1 ★,61,66,66,69,47,60,3,67+0,67+0,67+0,66+0,67+0,67+0,67+0,66+0,70+0,70+0,70+0,66+0,68+0,68+0,68+0,66+0,56+0,65+0,65+0,65+0,56+0,57+0,51+0,51+0,51+0,57+0,22+0,Male
4,41,Iniesta,36,81,Spain,Vissel Kobe,82,CAM,CM CAM,https://cdn.sofifa.com/players/000/041/20_120.png,https://cdn.sofifa.com/teams/101146/light_60.png,https://cdn.sofifa.com/flags/es.png,81,Vissel Kobe 2018 ~ 2021,"5'7""",150lbs,Right,0,"Jul 16, 2018",NaN,€5.5M,€12K,€7.2M,2018 ~ 2021,367,75,69,54,90,79.0,408,85,80.0,70,83,90,346,61,56,79.0,75,75.0,297,67,40.0,58,62,70,370,58,70.0,78.0,93.0,71,89.0,181,68,57,56.0,45,6,13,6,13,7,2014,420,4 ★,4★,High,Medium,4 ★,58,70,85,85,63,59,149,72+3,72+3,72+3,79+0,79+0,79+0,79+0,79+0,82+-1,82+-1,82+-1,79+2,81+0,81+0,81+0,79+2,70+3,73+3,73+3,73+3,70+3,67+3,64+3,64+3,64+3,67+3,17+3,Male


### Set Option Display max rows/columns (due to size of Dataset)

In [22]:
pd.set_option('display.max_rows', 110)
pd.set_option('display.max_columns', 110)

### Cleaning Column Names

In [23]:
list(data.columns)

['ID',
 'Name',
 'Age',
 'OVA',
 'Nationality',
 'Club',
 'BOV',
 'BP',
 'Position',
 'Player Photo',
 'Club Logo',
 'Flag Photo',
 'POT',
 'Team & Contract',
 'Height',
 'Weight',
 'foot',
 'Growth',
 'Joined',
 'Loan Date End',
 'Value',
 'Wage',
 'Release Clause',
 'Contract',
 'Attacking',
 'Crossing',
 'Finishing',
 'Heading Accuracy',
 'Short Passing',
 'Volleys',
 'Skill',
 'Dribbling',
 'Curve',
 'FK Accuracy',
 'Long Passing',
 'Ball Control',
 'Movement',
 'Acceleration',
 'Sprint Speed',
 'Agility',
 'Reactions',
 'Balance',
 'Power',
 'Shot Power',
 'Jumping',
 'Stamina',
 'Strength',
 'Long Shots',
 'Mentality',
 'Aggression',
 'Interceptions',
 'Positioning',
 'Vision',
 'Penalties',
 'Composure',
 'Defending',
 'Marking',
 'Standing Tackle',
 'Sliding Tackle',
 'Goalkeeping',
 'GK Diving',
 'GK Handling',
 'GK Kicking',
 'GK Positioning',
 'GK Reflexes',
 'Total Stats',
 'Base Stats',
 'W/F',
 'SM',
 'A/W',
 'D/W',
 'IR',
 'PAC',
 'SHO',
 'PAS',
 'DRI',
 'DEF',
 'PHY',
 

In [24]:
data.columns = data.columns.str.lower()
data.columns = [column.lower().replace(' ', '_').replace('/', '_') for column in data.columns]

In [25]:
data.columns

Index(['id', 'name', 'age', 'ova', 'nationality', 'club', 'bov', 'bp',
       'position', 'player_photo',
       ...
       'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk', 'gender'],
      dtype='object', length=107)

### Drop Columns

In [26]:
data.shape

(17125, 107)

**drop columns that cannot be used for analysis:** <br>
- name and id do not add any value to our model
- player_photo, club_logo, flag_photo are links
- there is only one gender so no added information
- loan_date_end are mainly null values or unstructured dates
- position because of several positions per row, we also have the column best position to use for position
- stats per position are dropped for now because I could not find a reason to keep them

In [27]:
data = data.drop(data.loc[:, "id":"name"].columns, axis = 1)
data = data.drop(['player_photo', "club_logo", "flag_photo", "gender", "loan_date_end", "position", "team_&_contract", "joined"], axis=1)
data = data.drop(data.loc[:, "ls":"gk"].columns, axis = 1)
data = data.drop(data.loc[:, "contract":"sliding_tackle"].columns, axis = 1)
data = data.drop(data.loc[:, "gk_diving":"base_stats"].columns, axis = 1)

In [28]:
data.shape

(17125, 27)

### Clean up Columns containing €,K,M Values

Some columns contain numerical values but also special characters that need to be removed for the model.

In [29]:
def clean_char(x):
    if (x.startswith('€') and x.endswith('K') and (x.find('.')>=0)):
        return x.replace("€","").replace("K","").replace(".","") + "00"      
    elif (x.startswith('€') and x.endswith('K')):
        return x.replace("€","").replace("K","") + "000" 
    if (x.startswith('€') and x.endswith('M') and (x.find('.')>=0)):
        return x.replace("€","").replace("M","").replace(".","") + "00000"      
    elif (x.startswith('€') and x.endswith('M')):
        return x.replace("€","").replace("M","") + "000000" 
    else:
        return x.replace("€","")

data['value'] = pd.to_numeric(data['value'].apply(clean_char))
data['wage'] = pd.to_numeric(data['wage'].apply(clean_char))
data['release_clause'] = pd.to_numeric(data['release_clause'].apply(clean_char))

### Clean up Columns containing height/weight

In [32]:
# converting lbs to kg and removing string in value

def clean_weight(x):
    if 'lbs' in str(x):
        x = str(x).replace('lbs','')
        x = float(x) * 0.454
    return float(x)

In [33]:
data['weight'] = pd.to_numeric(data['weight'].apply(clean_weight))

In [35]:
# converting inches to cm and removing string in value

data['height'] = data['height'].str.replace('""',' ')
data['height'] = data['height'].apply(lambda x: int(x.split("''")[0])*30.48 + int(x.split("''")[1])*2.54)
data['height']

ValueError: invalid literal for int() with base 10: '6\'0"'

### Review DTypes

In [34]:
data.dtypes

age                 int64
ova                 int64
nationality        object
club               object
bov                 int64
bp                 object
pot                 int64
height             object
weight            float64
foot               object
growth              int64
value               int64
wage                int64
release_clause     object
goalkeeping         int64
w_f                object
sm                 object
a_w                object
d_w                object
ir                 object
pac                 int64
sho                 int64
pas                 int64
dri                 int64
def                 int64
phy                 int64
hits               object
dtype: object

**changes of dtypes I want to perform:** <br>
- 

In [ ]:
data = data.astype({'bp':'boolean', 'sex':'boolean', 'smoking':'boolean', 'high_blood_pressure':'boolean','diabetes':'boolean','anaemia':'boolean','age':'int64'})

### Drop Duplicates

In [15]:
data.shape

(17125, 25)

In [16]:
data = data.drop_duplicates()

In [17]:
data.shape

(17125, 25)

No duplicates present in this dataset.

##  Review Cleaned Dataset

In [18]:
round(data.corr()['value'].sort_values(ascending=False),2)

KeyError: 'value'

In [ ]:
data.describe()

In [ ]:
data.corr()

In [ ]:
# create correlation heatmap 

mask = np.zeros_like(data.corr())

mask[np.triu_indices_from(mask)] = True

fig, ax = plt.subplots(figsize=(10, 8))
ax = sns.heatmap(data.corr(), mask=mask, vmin=-1, vmax=1, annot=True, cmap='BrBG')

plt.show()